# Crawling

## 에브리타임에서 강의평 수집하기

In [ ]:
from selenium import webdriver

from bs4 import BeautifulSoup

import time
import pandas as pd
import re

In [ ]:
data = pd.read_csv('../data/2020LectureTable.csv', index_col=0)
data

In [ ]:
lecture_urls = [] ; lecture_names = [] ; professor_names = [] ; 
review_rates = [] ; review_semesters = [] ; review_texts = []

In [ ]:
def GetAllReviews(lecture_url, driver):
    
    global lecture_urls, lecture_names, professor_names, review_rates, review_semesters, review_texts
    
    # html 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    lecture_name = soup.find('h2').text
    professor_name = soup.find(attrs={'class':'side head'}).find_all('span')[1].text

    try:
        # 강의평이 있는 경우
        articles = soup.find(attrs={'class':'articles'}).find_all('article')
        time.sleep(3)

        for article in articles:
            rate = article.find(attrs={'class':'rate'}).find(attrs={'class':'on'})['style']
            semester = article.find(attrs={'class':'info'}).find(attrs={'class':'semester'}).text
            text = article.find(attrs={'class':'text'}).text
            
            lecture_urls.append(lecture_url)
            lecture_names.append(lecture_name)
            professor_names.append(professor_name)
            review_rates.append(rate)
            review_semesters.append(semester)
            review_texts.append(text)
    
    except:
        # 강의평이 없는 경우
        lecture_urls.append(lecture_url)
        lecture_names.append(lecture_name)
        professor_names.append(professor_name)
        review_rates.append(None)
        review_semesters.append(None)
        review_texts.append(None)

In [ ]:
# Settings
LECTURE_URLS = list(set(data['LectureUrl']))
URL = 'https://yonsei.everytime.kr/lecture'
USER_ID = 'dlgyeka12'
USER_PW = 'power01'

In [ ]:
# 브라우저 생성 및 url 호출
driver = webdriver.Chrome('chromedriver')
driver.get(URL)

In [ ]:
# 로그인
driver.find_element_by_xpath('//*[@id="container"]/form/p[1]/input').send_keys(USER_ID)
driver.find_element_by_xpath('//*[@id="container"]/form/p[2]/input').send_keys(USER_PW)
driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

In [ ]:
time.sleep(5)
driver.find_element_by_xpath('//*[@id="sheet"]/ul/li[3]/a').click() # 팝업창 닫기

In [ ]:
for lecture_url in LECTURE_URLS:
    driver.get(lecture_url) # 브라우저 생성 및 url 호출
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 웹페이지 최하단까지 스크롤
    time.sleep(2)
    GetAllReviews(lecture_url, driver) # 데이터 수집

In [ ]:
# 브라우저 종료
driver.quit()

In [ ]:
# 데이터셋 생성
data = pd.DataFrame({'LectureUrl':lecture_urls, 'LectureName':lecture_names, 'LectureProf':professor_names, 
                     'ReviewStar':review_rates, 'ReviewSemester':review_semesters, 'ReviewText':review_texts})

In [ ]:
# 데이터셋 확인
data.info()

In [ ]:
# ReviewStar 후처리
data['ReviewStar'] = data['ReviewStar'].fillna('').apply(lambda x: re.findall('[0-9]+', x)[0] if x!='' else x)

In [ ]:
# 데이터셋 저장
data.to_csv('../data/2020LectureReview.csv')